In [1]:
!pip install transformers
!sudo apt-get install git-lfs
!git lfs install
!git clone https://huggingface.co/google-bert/bert-base-multilingual-cased




git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.
Error: Failed to call git rev-parse --git-dir: exit status 128 
Git LFS initialized.
Cloning into 'bert-base-multilingual-cased'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 61 (delta 21), reused 18 (delta 18), pack-reused 34 (from 1)
Unpacking objects: 100% (61/61), 1.38 MiB | 4.61 MiB/s, done.
Filtering content: 100% (4/4), 3.00 GiB | 104.76 MiB/s, done.


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import time
from datasets import Dataset
from sklearn import metrics
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding
import torch.nn.functional as F

In [3]:
df = pd.read_csv("../data/hatebr_and_rationales.csv")

TRAIN_SIZE = 0.8
TEST_SIZE = 0.1
VAL_SIZE = 0.1


x_train, x_test_val, y_train, y_test_val = train_test_split(df['normalized_text'], df['label final'], test_size=TEST_SIZE + VAL_SIZE, random_state=0)
x_test, x_val, y_test, y_val = train_test_split(x_test_val, y_test_val, test_size=VAL_SIZE/(TEST_SIZE + VAL_SIZE), random_state=0)

In [4]:
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-multilingual-cased')
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-multilingual-cased")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [5]:
model = AutoModelForSequenceClassification.from_pretrained('../data/mbert-fine-tuned')

In [6]:
def predict_fn(texts):
    
    if isinstance(texts, pd.Series):
        texts = texts.tolist()
    elif isinstance(texts, np.ndarray):
        texts = texts.tolist()
    elif isinstance(texts, str):
        texts = [texts]
        
        # Ensure that `texts` is a list of strings
    if not all(isinstance(text, str) for text in texts):
        raise ValueError("All elements in `texts` should be strings.")
    
    model.eval()
    
    # Tokenize the input texts
    encodings = tokenizer(texts, truncation=True, padding=True, max_length=512, return_tensors='pt')
    
    # Move tensors to the same device as the model (e.g., CPU or GPU)
    input_ids = encodings['input_ids'].to(model.device)
    attention_mask = encodings['attention_mask'].to(model.device)
    
    # Perform inference (forward pass)
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    
    # Get the logits (raw predictions)
    logits = outputs.logits
    
    # Apply softmax to get probabilities
    probs = F.softmax(logits, dim=-1)
    
    # Return probabilities as a numpy array (LIME and SHAP expect numpy arrays)
    return probs.cpu().numpy()

In [7]:
instances = np.where(y_train == 1)[0][0:350]

In [8]:
from lime.lime_text import LimeTextExplainer
import time
class_names = ["Non-hate", "Hate"]
explainer = LimeTextExplainer(class_names = class_names)

explainers = []
for i in instances:
    t0 = time.time()
    exp = explainer.explain_instance(x_train.iloc[i], predict_fn, num_features = 10, num_samples=200, labels=(1,)) 
    explainers.append(exp)
    print(time.time() - t0)

13.518953084945679
23.88949155807495
1.492811679840088
4.148914813995361
10.551529169082642
6.127687692642212
1.201331615447998
1.4755773544311523
6.742788314819336
1.7753586769104004
28.867960691452026
7.1461265087127686
5.568728923797607
2.2872750759124756
7.068715333938599
14.499767780303955
2.7077972888946533
14.554745435714722
3.154970645904541
4.04448390007019
14.81830883026123
4.314949989318848
4.574764251708984
2.031348705291748
6.0707666873931885
7.954641103744507
2.584965229034424
7.716684818267822
9.209814548492432
12.44246506690979
8.305974245071411
2.257577896118164
12.398608446121216
8.765159606933594
2.861361026763916
3.453019857406616
6.488985300064087
1.820810079574585
5.046830892562866
2.854905605316162
2.0845797061920166
7.904132843017578
1.807401418685913
6.771476745605469
11.907674074172974
10.439992904663086
4.525145769119263
3.3982126712799072
2.9009149074554443
8.19269847869873
2.3243491649627686
4.823625326156616
3.1256754398345947
10.46851110458374
16.27290320

In [9]:
with open('../results/results_lime_mbert_0_a_350.txt', 'w') as f:
    for exp in explainers:
        f.write(str(exp.as_list()))
        f.write('\n')